In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [5]:
#!pip install langchain==0.1.6

In [8]:
#!pip install sentence-transformers

In [7]:
#!pip install chromadb

In [10]:
#!pip install transformers

# Import LLM 

In [11]:
import warnings as wn
wn.filterwarnings('ignore')

In [12]:
import transformers

In [13]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextStreamer
from transformers import Pipeline,pipeline

2024-05-07 17:33:47.149831: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 17:33:47.149939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 17:33:47.314195: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [15]:
import torch

In [16]:
model_path='/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'

tokenizer = AutoTokenizer.from_pretrained(
    model_path
)


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    offload_folder='offload',
    device_map='auto',
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
streamer = TextStreamer(
    tokenizer=tokenizer
)

In [101]:
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 500,
#     streamer = streamer,
)

In [102]:
llm = HuggingFacePipeline(pipeline=pipe)

In [82]:
llm("Who is elon?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Who is elon?
User 1: Elon Musk is a billionaire entrepreneur who has founded multiple companies, including SpaceX, Tesla, Neuralink, and The Boring Company. He is known for his ambitious plans to revolutionize various industries, such as space travel, electric cars, and artificial intelligence. Musk has also been involved in several high-profile projects, including the development of the Hyperloop and the colonization of Mars.</s>


'\nUser 1: Elon Musk is a billionaire entrepreneur who has founded multiple companies, including SpaceX, Tesla, Neuralink, and The Boring Company. He is known for his ambitious plans to revolutionize various industries, such as space travel, electric cars, and artificial intelligence. Musk has also been involved in several high-profile projects, including the development of the Hyperloop and the colonization of Mars.'

# Use Langchain

In [103]:
# import packages from langchains for llm

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings

# Scrap Data from Website

In [104]:
# import python packages to scrap data from sites

from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader

In [149]:
web_path = 'https://www.upwork.com/freelancers/~015e7fe10fdf1c7f4f'

In [150]:
# now load the document

loader = WebBaseLoader(
    web_path = web_path,
    header_template = None,
    verify_ssl = True,
    proxies = None,
    continue_on_failure = False,
    requests_per_second = 2,
    default_parser = 'html.parser'
)

In [151]:
docs = loader.load()
txt_data = docs[0].page_content

In [152]:
len(txt_data)

57

In [153]:
print(txt_data[0:500])

Just a moment...Enable JavaScript and cookies to continue


# Text Splitter

In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

In [107]:
# now define the chunks and overlap

chunks_size = 500
chunks_overlap = 40

rec_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=chunks_size,
    chunk_overlap=chunks_overlap,
    is_separator_regex=False
)

In [92]:
rec_splitter.split_text(txt_data[0:1000])

['What is LangChain? Getting Started with LangChain | DataStaxProductsPricingStoriesResourcesDocsContact UsSign InSign InTry For FreeHANDS-ON WORKSHOP MAY 16: Learn to build a multimodal GenAI recommendation appBack to ResourcesGuide • Nov 09, 2023What is LangChain? Understanding Key Components and PrinciplesLangChain aids in implementing the Retrieval Augmented Generation (RAG) pattern in applications, simplifying data retrieval from various sources. How does this contribute to more contextually',
 'this contribute to more contextually aware AI applications?Sign Up for AstraDavid DierufAI Developer AdvocateWhat is LangChain?LangChain is a Python framework designed to streamline AI application development, focusing on real-time data processing and integration with Large Language Models (LLMs). It offers features for data communication, generation of vector embeddings, and simplifies the interaction with LLMs, making it efficient for AI developers.When you refer to an application as “AI

In [93]:
# now try the Simple charater splitter

splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=chunks_size,
    chunk_overlap = chunks_overlap
)

In [94]:
splitter.split_text(txt_data[0: 1500])

['What is LangChain? Getting Started with LangChain | DataStaxProductsPricingStoriesResourcesDocsContact UsSign InSign InTry For FreeHANDS-ON WORKSHOP MAY 16: Learn to build a multimodal GenAI recommendation appBack to ResourcesGuide • Nov 09, 2023What is LangChain? Understanding Key Components and PrinciplesLangChain aids in implementing the Retrieval Augmented Generation (RAG) pattern in applications, simplifying data retrieval from various sources. How does this contribute to more contextually aware AI applications?Sign Up for AstraDavid DierufAI Developer AdvocateWhat is LangChain?LangChain is a Python framework designed to streamline AI application development, focusing on real-time data processing and integration with Large Language Models (LLMs). It offers features for data communication, generation of vector embeddings, and simplifies the interaction with LLMs, making it efficient for AI developers.When you refer to an application as “AI” that usually means it includes interact

In [33]:
split = rec_splitter.split_documents(
    docs
)

In [34]:
embedding = HuggingFaceEmbeddings(
    model_name='all-MiniLM-L6-v2'
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
t1= "I love dogs"
t2 = "I love cats"
t3 = "Study is very hard"

In [36]:
embd1 = embedding.embed_query(t1)
embd2 = embedding.embed_query(t2)
embd3 = embedding.embed_query(t3)

In [37]:
np.dot(embd1,embd2), np.dot(embd2,embd3)

(0.7720984757242813, 0.06933481201584218)

# Add Vector Databases

In [38]:
from langchain.vectorstores import Chroma
from langchain.vectorstores import pinecone

In [40]:
#!chmod 664 /kaggle/working

In [39]:
db = Chroma.from_documents(
    documents= split,
    embedding=embedding,
    collection_name='langchain',
    persist_directory='docs/chroma/'
)

In [40]:
query = 'What is langchain?'

In [41]:
db.similarity_search(
    query=query,
    k=3
)

[Document(page_content='completion of the prompt.All the steps in the RAG pattern need to happen in succession. If anything errors along the way, the processing should stop. LangChain offers the “chain” construct to attach steps together in a specific way with a specific configuration. All LangChain libraries follow this chain construct to make it very easy to move steps around and create powerful processing pipelines.What are the Benefits of Using LangChain?Using LangChain offers several benefits in the realm of', metadata={'description': 'In this blog, we cover LangChain: what it is, how it works, examples, & its ability to enhance applications by combining LLMs with other computational sources.', 'language': 'en', 'source': 'https://www.datastax.com/guides/what-is-langchain', 'title': 'What is LangChain? Getting Started with LangChain | DataStax'}),
 Document(page_content='in LangChain are the decision-making components. They determine the best course of action based on the input, t

In [42]:
# using MMR

mmr_docs = db.max_marginal_relevance_search(
    query=query,
    k=2,
    fetch_k=2
)

In [43]:
mmr_docs

[Document(page_content='completion of the prompt.All the steps in the RAG pattern need to happen in succession. If anything errors along the way, the processing should stop. LangChain offers the “chain” construct to attach steps together in a specific way with a specific configuration. All LangChain libraries follow this chain construct to make it very easy to move steps around and create powerful processing pipelines.What are the Benefits of Using LangChain?Using LangChain offers several benefits in the realm of', metadata={'description': 'In this blog, we cover LangChain: what it is, how it works, examples, & its ability to enhance applications by combining LLMs with other computational sources.', 'language': 'en', 'source': 'https://www.datastax.com/guides/what-is-langchain', 'title': 'What is LangChain? Getting Started with LangChain | DataStax'}),
 Document(page_content='in LangChain are the decision-making components. They determine the best course of action based on the input, t

# Compressor Technique

In [39]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [48]:
# meta_data_field_info = [
#     AttributeInfo(
#         name='source',
#         description='Text chunks should be one of `/docs/use_cases/query_analysis/`',
#         type='string'
#     ),
#     AttributeInfo(
#         name='source',
#         description='The heading from contexts',
#         type='string'
#     )
# ]

In [43]:
# docs_content_disc = 'Langchain RAG'

# ret = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     docs_content_disc,
#     meta_data_field_info,
#     verbose=True
# )

In [49]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [50]:
compressor = LLMChainExtractor.from_llm(
    llm=llm
)

In [51]:
compressor_ret = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=db.as_retriever(
        search_type = 'mmr'
    )
)

In [52]:
question = "What is the langchain?"

In [54]:
# compressed_docs = compressor_ret.get_relevant_documents(
#     query=question
# )

In [55]:
pretty_print_docs(compressed_docs)

NameError: name 'compressed_docs' is not defined

# LLM + RAG

In [108]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [54]:
# this function is for clear chromadb

import shutil

def delete_all(directory):
    try:
        shutil.rmtree(directory)
        print(f"All files and subdirectories in {directory} have been deleted.")
    except Exception as e:
        print(f"Failed to delete {directory}: {e}")

# Usage
directory_path = '/kaggle/working/docs/'
delete_all(directory_path)

All files and subdirectories in /kaggle/working/docs/ have been deleted.


In [138]:
# Create prompt template for LLM

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer. 

{context}
Question: {question}
Helpful Answer:"""

In [139]:
qa_chain_prompt = PromptTemplate.from_template(
    template=template
)

# Add memory

In [140]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

In [141]:
mem = ConversationBufferMemory(
     memory_key='chat_history', 
    return_messages=True, 
    human_prefix='Human',
    ai_prefix='AI',
    input_key='question',
    verbose=False,
)
window_mem = ConversationBufferWindowMemory(
    k=5,
    memory_key='chat_history', 
    return_messages=True, 
    output_key='answer',
    verbose=False
)

In [142]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = db.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 3, 'fetch_k': 50}
    ),
    return_source_documents=False,
    chain_type_kwargs={
        "prompt": qa_chain_prompt,
        "verbose": False,
        "memory": mem
    }
)

In [154]:
question = ""

In [155]:
result = qa_chain.invoke({'query':question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [156]:
print(result['result'])

 llamaIndex is an open-source AI application that uses a neural network in real-time to process data and provide intelligent responses to user queries. It is built using the RAG pattern and includes a pipeline that involves creating an embedding from the user's question, looking up context from a vector database, creating a prompt, and submitting it to a large language model (LLM) for completion. Thanks for asking!


In [79]:
while True:
    question = input("Ask: ")
    result = qa_chain.invoke({'query':question})
    print(result["result"])
    print('\n')

Ask:  What we need this framework?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 We need this framework because it is accurate and versatile. It can be used across various sectors and provides a lot of power by being able to handle diverse language tasks.




Ask:  Can you give me any code example if avaible?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 I'm sorry, I don't have any code examples available. However, I can provide you with more information about LangChain and its capabilities. LangChain is a framework that can be used to build generative AI applications. It offers a few different scenarios that implement the Retrieval-Augmented Generation (RAG) pattern, such as a simple question/answer example, a RAG over code example, and a Chat History example. Output parsers make quick work of this by offering a deep integration between how the LLM will respond and the application’s custom classes. Learn more about output parsers. When a model is created, it’s trained with some data set. A model is considered small or large based on the amount of data that was used during training. For example, LLMs like GPT have been trained on the entire public internet. That amount of data deserves the “Large” label. The challenge is that no matter how large the model, it can still be limited by the quality and quantity of the training data. To ad

KeyboardInterrupt: Interrupted by user

In [131]:
print(mem.buffer)

[HumanMessage(content='What is important features in langchain. tell me in bullets.'), AIMessage(content='\n\n* Communicating with Models: LangChain allows you to communicate with models in real-time, without the need for manual input or note-taking.\n* Contextual Awareness: LangChain can understand the context of the conversation and adjust its responses accordingly, making it more productive and enjoyable to use.\n* Integration with Popular Tools: LangChain can integrate with popular tools such as Apache, Apache Cassandra, Cassandra, Apache Pulsar, and Pulsar, making it easy to use and deploy.\n\nThanks for asking!'), HumanMessage(content='What is LAW Degree?'), AIMessage(content=' A LAW degree is a legal degree that is typically earned by students who want to become lawyers. It is a graduate degree that is offered by law schools and is designed to provide students with the knowledge and skills they need to practice law. A LAW degree typically includes courses in subjects such as con